In [ ]:
# default_exp funcs.gen_train_funcs

In [ ]:
%load_ext autoreload
%autoreload 2

# Training loop simulator learning

In [ ]:
# export
from decode_fish.imports import *
from decode_fish.funcs.evaluation import *
from decode_fish.funcs.file_io import *
from decode_fish.funcs.emitter_io import *
from decode_fish.funcs.utils import *
from decode_fish.funcs.dataset import *
from decode_fish.funcs.output_trafo import *
from decode_fish.funcs.plotting import *
from decode_fish.funcs.predict import *
import torch.nn.functional as F
from torch import distributions as D
from torch.utils.data import DataLoader
import torch_optimizer
from decode_fish.engine.microscope import Microscope, get_roi_filt_inds, extract_psf_roi, mic_inp_apply_inds, add_pos_noise, concat_micro_inp
from decode_fish.engine.model import UnetDecodeNoBn
from decode_fish.engine.point_process import PointProcessUniform, get_phased_ints
from decode_fish.engine.gmm_loss import PointProcessGaussian
import shutil
import wandb
import kornia

from hydra import compose, initialize
from decode_fish.funcs.merfish_eval import *
from decode_fish.funcs.exp_specific import *
# from decode_fish.funcs.visualization vimport get_simulation_statistics

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [ ]:
#export 
def gen_train(cfg,
          model, 
          microscope,
          post_proc,
          dl, 
          optim_dict):
    
    save_dir = Path(cfg.output.save_dir)
    model.cuda().train()

    # Save initial psf state
    torch.save(microscope.psf.state_dict(), str(save_dir) + '/psf_init.pkl' )

    # Load codebook 
    if 'codebook' in cfg:
        code_ref, targets = hydra.utils.instantiate(cfg.codebook)

    # Controls which genmodel parameters are optimized
    for name, p in microscope.named_parameters():
        p.requires_grad = cfg.training.mic.par_grads[name]

    calc_log_p_x = False
    
    ret_dict = next(iter(dl))
    x, local_rate, background = ret_dict['x'], ret_dict['local_rate'], ret_dict['background'] 
    zcrop, ycrop, xcrop = ret_dict['crop_z'], ret_dict['crop_y'], ret_dict['crop_x']

    background = background * microscope.get_ch_mult().detach()
    x = x * microscope.get_ch_mult().detach()

    colshift_crop = get_color_shift_inp(microscope.color_shifts, microscope.col_shifts_yx, ycrop, xcrop, cfg.sim.random_crop.crop_sz)

    with torch.no_grad():

        out_col = []
        bg_pred = []
        for i in range(len(x)):
            out_inp = model.tensor_to_dict(model(torch.concat([x[i:i+1], colshift_crop[i:i+1]], 1)))
            out_col.append(post_proc.get_micro_inp(out_inp))
            bg_pred.append(out_inp['background'])

        bg_pred = torch.concat(bg_pred)
        proc_out_inp = concat_micro_inp(out_col, batch_size=1)

        ch_out_inp = microscope.get_single_ch_inputs(*proc_out_inp, ycrop=ycrop.flatten(), xcrop=xcrop.flatten())
        filt_inds = get_roi_filt_inds(*ch_out_inp[0], microscope.psf.psf_volume.shape, x.shape, slice_rec=cfg.genm.exp_type.slice_rec, min_dist=10)
        print(len(ch_out_inp[1]), len(filt_inds))
        
        ch_out_inp = mic_inp_apply_inds(*ch_out_inp, filt_inds)

        rois = extract_psf_roi(ch_out_inp[0], x, torch.tensor([len(ch_out_inp[1]), 1,1,21,21]))
        bgs = extract_psf_roi(ch_out_inp[0], bg_pred, torch.tensor([len(ch_out_inp[1]), 1,1,21,21]))
        
        if cfg.training.mic.edge_diff:
        
            bg_edges = torch.cat([bgs[:,0,0,0,:], bgs[:,0,0,-1,:], bgs[:,0,0,:,0], bgs[:,0,0,:,-1]], 1)
            rois_edges = torch.cat([rois[:,0,0,0,:], rois[:,0,0,-1,:], rois[:,0,0,:,0], rois[:,0,0,:,-1]], 1)
            edge_diff = rois_edges.mean(-1) - bg_edges.mean(-1)
            rois -= edge_diff[:,None,None,None,None]
        
        ch_inds = ch_out_inp[0][1]
        int_vals = ch_out_inp[-2]
        
        del(model)
        del(out_col)
        free_mem()

    for batch_idx in range(cfg.training.start_iter, cfg.training.num_iters+1):

        optim_dict['optim_mic'].zero_grad()
        calc_log_p_x = False

#         int_means = torch.ones(cfg.genm.exp_type.n_channels).cuda() * (model.int_dist.int_loc.detach() + model.int_dist.int_conc.detach())
#         for i in range(cfg.genm.exp_type.n_channels):
#             if i in ch_inds:
#                 int_means[i] = int_vals[ch_inds == i].mean()

#         int_means = (model.int_dist.int_loc.detach() + model.int_dist.int_conc.detach()) / int_means
#         ch_fac_loss = torch.sqrt(torch.mean((microscope.channel_facs - microscope.channel_facs.detach() * int_means)**2))

        ch_out_inp = microscope.get_single_ch_inputs(*proc_out_inp, ycrop=ycrop.flatten(), xcrop=xcrop.flatten())
        ch_out_inp = mic_inp_apply_inds(*ch_out_inp, filt_inds)
        
        psf_n, psf_recs = microscope(*ch_out_inp, ret_psfs=True, add_noise=False)  
        
        mean_diff = 0.
        if cfg.training.mic.mean_diff:
            mean_diff = rois.mean([1,2,3,4], keepdim=True) - (psf_recs.detach()+bgs).mean([1,2,3,4], keepdim=True)

        log_p_x_given_z = -microscope.noise(psf_recs, bgs, const_theta_sim=False, ch_inds=ch_out_inp[0][1]).log_prob((rois-mean_diff).clamp_min_(1.))
        if cfg.training.mic.psf_loss_mult:
            log_p_x_given_z = log_p_x_given_z*psf_n.detach()
        log_p_x_given_z = log_p_x_given_z.mean()
        calc_log_p_x = True

#         log_p_x_given_z += ch_fac_loss

        if cfg.training.mic.norm_reg:
            log_p_x_given_z += cfg.training.mic.norm_reg * (microscope.psf.com_loss())

        if cfg.training.mic.l1_reg:
            log_p_x_given_z += cfg.training.mic.l1_reg * (microscope.psf.l1_diff_norm(microscope.psf_init_vol))  

        log_p_x_given_z.backward()
        if cfg.training.mic.grad_clip:
            torch.nn.utils.clip_grad_norm_(microscope.parameters(), max_norm=cfg.training.mic.grad_clip, norm_type=2)

        optim_dict['optim_mic'].step()
        optim_dict['sched_mic'].step()

        # Logging
        if batch_idx % cfg.output.log_interval == 0:

#             print(batch_idx, log_p_x_given_z)
            wandb.log({'AE Losses/p_x_given_z': log_p_x_given_z.detach().cpu()}, step=batch_idx)
            wandb.log({'AE Losses/RMSE(rec)': torch.sqrt((((rois-mean_diff)-(psf_recs+bgs))**2).mean()).detach().cpu()}, step=batch_idx)
            wandb.log({'AE Losses/sum(psf)': F.relu(microscope.psf.psf_volume)[0].sum().detach().cpu()}, step=batch_idx)
            
            torch.save(microscope.state_dict(), save_dir/'microscope.pkl')

    wandb.finish()    

In [ ]:
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_MOp/sweep_mop_17/phasing:0.0//train.yaml')
cfg.run_name = 'gensweep_mop_1'

cfg.output.log_interval = 20
cfg.training.bs = 100
cfg.genm.microscope.norm='none'

cfg.training.mic.psf_loss_mult = False
cfg.training.net.enabled = False

# cfg.training.mic.par_grads.channel_facs = False
# cfg.training.mic.par_grads.channel_shifts = False
# cfg.training.mic.par_grads.psf_vol = True
cfg.training.mic.par_grads.theta_par = False
cfg.training.mic.par_grads.z_facs = False
cfg.training.mic.edge_diff = False
cfg.training.mic.mean_diff = False

cfg.data_path.model_init = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//models/fishcod/MERFISH_MOp/sweep_mop_12/num_iters:9995//'
cfg.training.num_iters = 5000
cfg.training.schedule = None
cfg.training.mic.enabled = True

In [ ]:
fname = f'../config/experiment/{cfg.run_name}.yaml'
OmegaConf.save(cfg, fname)
prepend_line(fname, '# @package _global_')
cfg.run_name

'gensweep_mop_1'

In [ ]:
cfg.training.mic.par_grads.channel_facs = True
cfg.training.mic.par_grads.channel_shifts = True
cfg.training.mic.par_grads.theta_par = True
cfg.training.mic.par_grads.psf_vol = True
cfg.training.mic.par_grads.color_shifts = True

In [ ]:
psf, noise, micro = load_psf_noise_micro(cfg)
post_proc = hydra.utils.instantiate(cfg.post_proc_isi)

In [ ]:
img_3d, decode_dl = get_dataloader(cfg)

Crop size larger than volume in at least one dimension. Crop size changed to (1, 56, 56)
7 volumes


/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
inp_offset, inp_scale = get_forward_scaling(img_3d[0])
# inp_scale = 300
# inp_offset = 100
model = hydra.utils.instantiate(cfg.network, inp_scale=inp_scale, inp_offset=inp_offset)

psf  .to('cuda')
model.to('cuda')
micro.to('cuda')

Microscope(
  (psf): LinearInterpolatedPSF(
    (forward_nonlin): Identity()
  )
  (noise): sCMOS()
)

In [ ]:
params = []
for param in model.parameters():
    params.append(param)

In [ ]:
optim_dict = {}
optim_dict['optim_mic'] = hydra.utils.instantiate(cfg.training.mic.opt, params=micro.parameters())
optim_dict['sched_mic'] = hydra.utils.instantiate(cfg.training.mic.sched, optimizer=optim_dict['optim_mic'])
    
save_dir = Path(cfg.output.save_dir)
save_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
# Resume training from a previous checkpoint
if cfg.data_path.micro_init is not None:
    micro.load_state_dict(torch.load(cfg.data_path.micro_init), strict=False)
    
# cfg.data_path.model_init = cfg.output.save_dir
# cfg.data_path.model_init = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//models/fishcod/MERFISH_MOp/sweep_mop_5b/int_loc:0.1xscale:400xpsf_noise:0.0'

if cfg.data_path.model_init is not None:
    print('loading')
    model = load_model_state(model, Path(cfg.data_path.model_init)/'model.pkl').cuda()
#     micro.load_state_dict(torch.load(Path(cfg.data_path.model_init)/'microscope.pkl'), strict=False)
    
    train_state_dict = torch.load(Path(cfg.data_path.model_init)/'training_state.pkl')
    for k in optim_dict:
        optim_dict[k].load_state_dict(train_state_dict[k])

loading


ValueError: loaded state dict contains a parameter group that doesn't match the size of optimizer's group

In [ ]:
code_ref, targets = hydra.utils.instantiate(cfg.codebook)
post_proc.codebook = torch.tensor(code_ref)

In [ ]:
_ = wandb.init(project=cfg.output.project, 
               config=OmegaConf.to_container(cfg, resolve=True),
               dir=cfg.output.log_dir,
               group=cfg.output.group,
               name=cfg.run_name,
               mode='disabled'
          )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [ ]:
gen_train(cfg=cfg,
     model=model, 
     microscope=micro, 
     post_proc=post_proc,
     dl=decode_dl, 
     optim_dict=optim_dict)

2292 540


KeyboardInterrupt: 

In [ ]:
!nbdev_build_lib